In [17]:
import json

In [26]:
import re

def has_latitude(text: str) -> bool:
    has_latitude = False
    matched_latitudes = re.findall(r"-?(?<!\d)\d{,2}(?<=\d)\.\d{,20}", text)

    if any([float(lat) >= -90 and float(lat) <= 90 for lat in  matched_latitudes]):
        has_latitude = True

    return has_latitude


def has_longitude(text: str) -> bool:
    has_latitude = False
    matched_latitudes = re.findall(r"-?(?<!\d)\d{,3}(?<=\d)\.\d{,20}", text)

    if any([float(lat) >= -180 and float(lat) <= 180 for lat in  matched_latitudes]):
        has_latitude = True

    return has_latitude


def has_coordinates(text: str) -> bool:
    has_lat = has_latitude(text)
    has_lon = has_longitude(text)

    if has_lat and has_lon:
        return True
    else:
        return False

CONTENT_TYPES = [
    "text/csv",
    "text/html",
    # "text/javascript",
    "text/xml",
    "text/plain",
    "application/json",
    "application/xml",
    "application/zip",
    "application/x-www-form-urlencoded",
    "multipart/form-data",
]

url_stop_words = [
    "css",
    "styles",
    "google",
    "googleapis",
    "bing",
    "virtualearth",
    "autosuggest",
    "geocode",
    "mapbox",
    "tomtom",
    "gstatic",
]

In [2]:
with open("../har_examples/www.avoska.ru.har") as file:
    har = json.load(file)

In [4]:
entries = har["log"]["entries"]

In [31]:
def filter_requests(entries):
    for request in entries:
        status = request["response"]["status"]

        if status == 200:
            if not any([stop_word in request["request"]["url"] for stop_word in url_stop_words]):
                content = request["response"]["content"]

                if content.get("size") > 0 and len(content.get("text", "")) > 0 and content["mimeType"] in CONTENT_TYPES:
                    if has_coordinates(content.get("text")):
                        yield request["request"]["url"]
                    

list(filter_requests(entries))

['https://www.avoska.ru/shops/',
 'https://www.avoska.ru/api/get_shops.php?',
 'https://www.avoska.ru/api/get_shops.php?map=1']